# PageRank

## Обработка данных

In [1]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
from typing import Tuple, Mapping, Any

In [4]:
df = pd.read_csv("res\\movie\\rating.csv")

In [5]:
df['rating'].describe()

count    2.000026e+07
mean     3.525529e+00
std      1.051989e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [6]:
def split_by_target_column_value(df: pd.DataFrame, test_size: float, target_column: str, target_value: Any,
                                 random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Разделяет DataFrame на обучающий и тестовый наборы на основе определенного значения целевого столбца.
    Выдает в качестве тестового набора test_size от набора, в котором значение в target_column == target_value
    В тренировочном все остальное
    Параметры:
        df (pd.DataFrame): DataFrame для разделения.
        test_size: Пропорция данных, которые следует включить в тестовый набор.
        target_column (str): Название целевого столбца для разделения.
        target_value (Any): Значение целевого столбца, используемое для разделения.
        random_state (int): Зерно для генератора случайных чисел.

    Возвращает:
        Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]: Данные для обучения; тестовые данные для целевой колонки, которые не попали в data,
        тренировочные данные для целевой колоники, которые попали в data
    """
    grouped = df.groupby(by=target_column)
    train_df, test_df = train_test_split(grouped.get_group(target_value), test_size=test_size,
                                         random_state=random_state)
    data = pd.concat([group for ind, group in grouped if ind != target_value], ignore_index=True)
    data = pd.concat([train_df, data], ignore_index=True)
    return data, test_df, train_df



In [7]:
data, test1, train1 = split_by_target_column_value(df, test_size=0.5, target_column='userId', target_value=1)

In [8]:
data.head()

,userId,movieId,rating,timestamp
0,1,2288,4.0,2004-09-10 03:14:37
1,1,2648,3.5,2004-09-10 03:13:37
2,1,1222,3.5,2005-04-02 23:30:37
3,1,3037,3.5,2005-04-02 23:46:03
4,1,2683,3.5,2004-09-10 03:07:30


In [9]:
test1.head()

,userId,movieId,rating,timestamp
156,1,7045,3.5,2005-04-02 23:50:34
145,1,6242,3.5,2004-09-10 03:13:24
101,1,3081,4.0,2004-09-10 03:15:43
127,1,4911,4.0,2005-04-02 23:52:41
141,1,5898,3.5,2005-04-02 23:53:22


In [10]:
train1.head()

,userId,movieId,rating,timestamp
79,1,2288,4.0,2004-09-10 03:14:37
84,1,2648,3.5,2004-09-10 03:13:37
39,1,1222,3.5,2005-04-02 23:30:37
100,1,3037,3.5,2005-04-02 23:46:03
86,1,2683,3.5,2004-09-10 03:07:30


## Реализация PageRank

In [11]:
def get_range(data: pd.DataFrame, personalization: Mapping,
               source: str, target: str, edge_attr: str) -> pd.DataFrame:
    """
    Вычисляет PageRank на основе заданного DataFrame и возвращает DataFrame с рангами каждого фильма с учетом персонализации.
    Параметры:
        data (pd.DataFrame): DataFrame для вычисления PageRank.
        personalization (Mapping): Словарь с персонализацией для PageRank.
        source (str): Название столбца, содержащего исходные вершины ребер.
        target (str): Название столбца, содержащего целевые вершины ребер.
        edge_attr (str): Название столбца, содержащего атрибуты ребер.

    Возвращает:
        pd.DataFrame: DataFrame с результатами вычисления PageRank с учетом персонализации.
    """
    G = nx.Graph()

    # Создаем список ребер для добавления
    edges_to_add = [((row[source], 'source'), (row[target], 'target'), {'weight': row[edge_attr]}) for _, row in data.iterrows()]

    # Добавляем все ребра скопом
    G.add_edges_from(edges_to_add)
    
    # Преобразование персонализации в формат, подходящий для PageRank
    new_personalization = {(k, 'source'): v for k, v in personalization.items()}
    
    # Вычисление PageRank
    pagerank = nx.pagerank(G, personalization=new_personalization)
    
    # Формирование DataFrame с результатами PageRank
    target_rank_generator = ((node[0], rank) for node, rank in pagerank.items() if node[1] == 'target')
    return pd.DataFrame(target_rank_generator, columns=[target, 'pagerank'])


## Применение PageRank

In [12]:
# Создаем тестовый DataFrame
test_df = pd.DataFrame({
    'source': [1, 2, 2, 2],
    'target': [2, 2, 3, 1],
    'edge_attr': [5, 5, 5, 1]
})

# Создаем тестовую персонализацию
personalization = {1: 1,}

prediction_test = get_range(test_df, personalization=personalization, source='source', target='target',
                                            edge_attr='edge_attr')
prediction_test

,target,pagerank
0,2,0.346707
1,3,0.093959
2,1,0.018792


In [13]:
prediction_movie_for_user1 = get_range(data=data, personalization={1: 1}, source='userId', target='movieId',
                                            edge_attr='rating')


In [14]:
prediction_movie_for_user1.head()

,movieId,pagerank
0,2288,0.001772
1,2648,0.001459
2,1222,0.001858
3,3037,0.001467
4,2683,0.001792


In [15]:
prediction_movie_for_user1_without_famous = prediction_movie_for_user1[~(prediction_movie_for_user1['movieId'].isin(train1['movieId']))]
print(len(prediction_movie_for_user1_without_famous))
prediction_movie_for_user1_without_famous.head()

26657


,movieId,pagerank
87,3,0.000179
88,62,0.000319
89,70,0.000226
90,110,0.001070
91,242,0.000017


In [16]:
# Топ фильмов по мнению pagerank
pred = prediction_movie_for_user1_without_famous.sort_values(by='pagerank', ascending=False).head(len(test1))
print(len(pred))
pred.head()

88


,movieId,pagerank
592,296,0.001383
153,318,0.001371
305,356,0.001286
92,260,0.001196
263,2571,0.001149


In [17]:
def accuracy_without_order(true_labels, predicted_labels):
    true_set = set(true_labels)
    pred_set = set(predicted_labels)
    
    # Проверка на равенство множеств
    accuracy = len(true_set.intersection(pred_set)) / len(true_set)
    
    return accuracy

In [18]:
print(len(test1['movieId']))
test1['movieId'].head()

88


156    7045
145    6242
101    3081
127    4911
141    5898
Name: movieId, dtype: int64

In [19]:
print(len(pred['movieId']))
pred['movieId'].head()

88


592     296
153     318
305     356
92      260
263    2571
Name: movieId, dtype: int64

In [20]:
print(accuracy_without_order(test1['movieId'], pred['movieId']))

0.29545454545454547


## Интерпритация результата

In [21]:
movie_df = pd.read_csv('res/movie.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'res/movie.csv'

In [ ]:
prediction_moviename = pd.merge(movie_df, pred, on='movieId', how='inner')
prediction_moviename.sort_values(by='pagerank', ascending=False).head()

,movieId,title,genres,pagerank
11,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0.001383
13,318,"Shawshank Redemption, The (1994)",Crime|Drama,0.001371
16,356,Forrest Gump (1994),Comedy|Drama|Romance|War,0.001286
9,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,0.001196
71,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,0.001149


In [ ]:
test_moviename = pd.merge(movie_df, test1, on='movieId', how='inner')
print(len(test_moviename))
test_moviename.sort_values(by='rating', ascending=False).head()

88


,movieId,title,genres,userId,rating,timestamp
73,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,1,5.0,2005-04-02 23:30:19
87,8636,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,1,4.5,2005-04-02 23:44:53
21,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1,4.5,2005-04-02 23:30:24
20,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,1,4.5,2005-04-02 23:32:22
54,3499,Misery (1990),Drama|Horror|Thriller,1,4.0,2005-04-02 23:35:18


In [ ]:
# Фильмы которые были угаданны
test_moviename[test_moviename['movieId'].isin(prediction_moviename['movieId'])]

,movieId,title,genres,userId,rating,timestamp
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,1,3.5,2005-04-02 23:33:39
2,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1,3.5,2005-04-02 23:29:40
4,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1,4.0,2005-04-02 23:33:46
5,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller,1,4.0,2005-04-02 23:31:43
6,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1,4.0,2005-04-02 23:32:47
7,318,"Shawshank Redemption, The (1994)",Crime|Drama,1,4.0,2005-04-02 23:33:18
8,541,Blade Runner (1982),Action|Sci-Fi|Thriller,1,4.0,2005-04-02 23:30:03
9,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,1,3.5,2005-04-02 23:45:57
11,919,"Wizard of Oz, The (1939)",Adventure|Children|Fantasy|Musical,1,3.5,2004-09-10 03:07:01
12,1036,Die Hard (1988),Action|Crime|Thriller,1,4.0,2005-04-02 23:44:40
